In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm_notebook

from matplotlib import pyplot as plt

import mlflow
import mlflow.sklearn
import mlflow.lightgbm

mlflow.set_experiment("Sales Forecasting Models")

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

def add_date_features(df):
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofweek'] = df['Date'].dt.dayofweek
    return df

train = add_date_features(train)
test = add_date_features(test)


pairs = train[['store', 'product']].drop_duplicates()
results = []
prediction_test = []

for _, row in tqdm_notebook(pairs.iterrows()):
    store_id, product_id = row['store'], row['product']

    # Filter for that pair
    train_pair = train[(train['store'] == store_id) & (train['product'] == product_id)]
    test_pair = test[(test['store'] == store_id) & (test['product'] == product_id)]

    # Sort by Date
    train_pair = train_pair.sort_values("Date")

    # Time-based split: last 20% for validation
    cutoff = int(len(train_pair) * 0.8)
    train_data = train_pair.iloc[:cutoff]
    val_data = train_pair.iloc[cutoff:]

    X_train = train_data[['month', 'year', 'dayofweek']]
    y_train = train_data['number_sold']
    X_val = val_data[['month', 'year', 'dayofweek']]
    y_val = val_data['number_sold']

    X_test = test_pair[['month', 'year', 'dayofweek']]
    y_test = test_pair['number_sold']
    
    X_train_scaled = X_train.copy()
    X_val_scaled = X_val.copy()
    X_test_scaled = X_test.copy()

    X_train_scaled[['month', 'dayofweek']] = X_train_scaled[['month', 'dayofweek']].astype(str)
    X_val_scaled[['month', 'dayofweek']] = X_val_scaled[['month', 'dayofweek']].astype(str)
    X_test_scaled[['month', 'dayofweek']] = X_test_scaled[['month', 'dayofweek']].astype(str)

    X_train_scaled = pd.get_dummies(X_train_scaled, dtype=int)
    X_val_scaled = pd.get_dummies(X_val_scaled, dtype=int)
    X_test_scaled = pd.get_dummies(X_test_scaled, dtype=int)

    scaler = MinMaxScaler()

    scaler.fit(X_train_scaled)

    X_train_scaled.iloc[:,:] = scaler.transform(X_train_scaled)
    X_val_scaled.iloc[:,:] = scaler.transform(X_val_scaled)
    X_test_scaled.iloc[:,:] = scaler.transform(X_test_scaled)

    # -------- Ridge Regression + GridSearch --------
    
    with mlflow.start_run(run_name=f"Ridge_GS_store_{store_id}_prod_{product_id}"):
        ridge_pipeline = Pipeline([('ridge', Ridge())])
        ridge_param_grid = {
            'ridge__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
        }

        ridge_gs = GridSearchCV(ridge_pipeline, ridge_param_grid, cv=3)
        ridge_gs.fit(X_train_scaled, y_train)
        y_val_pred_gd_ridge = ridge_gs.predict(X_val_scaled)
        y_test_pred_gd_ridge = ridge_gs.predict(X_test_scaled)
        ridge_rmse_gd_val = mean_squared_error(y_val, y_val_pred_gd_ridge, squared=False)
        ridge_rmse_gd_test = mean_squared_error(y_test, y_test_pred_gd_ridge, squared=False)
        test_pair['ridge_gd'] = y_test_pred_gd_ridge
        
        # Log parameters and metrics
        mlflow.set_tag("model", "Ridge_GridSearch")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)
        mlflow.log_params(ridge_gs.best_params_)
        mlflow.log_metric("val_rmse", ridge_rmse_gd_val)
        mlflow.log_metric("test_rmse", ridge_rmse_gd_test)
        
        mlflow.sklearn.log_model(ridge_gs.best_estimator_, artifact_path="model")
        
        pd.DataFrame({'pred': y_test_pred_gd_ridge}).to_csv("y_test_pred_gd_ridge.csv", index=False)
        mlflow.log_artifact("y_test_pred_gd_ridge.csv")
        
    
    # -------- Ridge Regression + Hyperopt --------
    
    with mlflow.start_run(run_name=f"Ridge_HO_store_{store_id}_prod_{product_id}"):
        
        def ridge_objective(params):
            model = Ridge(alpha=params['alpha'])
            model.fit(X_train_scaled, y_train)
            preds = model.predict(X_val_scaled)
            rmse = mean_squared_error(y_val, preds, squared=False)
            return {'loss': rmse, 'status': STATUS_OK}

        ridge_space = {
            'alpha': hp.loguniform('alpha', np.log(1e-3), np.log(1e3))
        }

        ridge_trials = Trials()
        best_ridge_params = fmin(
            fn=ridge_objective,
            space=ridge_space,
            algo=tpe.suggest,
            max_evals=30,
            trials=ridge_trials,
            rstate=np.random.default_rng(42)
        )

        ridge_model = Ridge(alpha=best_ridge_params['alpha'])
        ridge_model.fit(X_train_scaled, y_train)
        y_val_pred_ho_ridge = ridge_model.predict(X_val_scaled)
        y_test_pred_ho_ridge = ridge_model.predict(X_test_scaled)
        ridge_rmse_ho_val = mean_squared_error(y_val, y_val_pred_ho_ridge, squared=False)
        ridge_rmse_ho_test = mean_squared_error(y_test, y_test_pred_ho_ridge, squared=False)
        test_pair['ridge_ho'] = y_test_pred_ho_ridge
        
        # Log parameters and metrics
        mlflow.set_tag("model", "Ridge_Hyperopt")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)
        mlflow.log_params(ridge_gs.best_params_)
        mlflow.log_metric("val_rmse", ridge_rmse_ho_val)
        mlflow.log_metric("test_rmse", ridge_rmse_ho_test)
        
        mlflow.sklearn.log_model(ridge_model, artifact_path="model")
        
        pd.DataFrame({'pred': y_test_pred_ho_ridge}).to_csv("y_test_pred_ho_ridge.csv", index=False)
        mlflow.log_artifact("y_test_pred_ho_ridge.csv")
    
    # -------- Random Forest + GridSearch --------
    
    with mlflow.start_run(run_name=f"RF_GS_store_{store_id}_prod_{product_id}"):
        
        rf_param_grid = {
            'n_estimators': [50, 100, 150],
            'max_depth': [5, 10, 15],
            'min_samples_split': [2, 5],
            'min_samples_leaf': [1, 2]
        }

        rf_grid = GridSearchCV(
            RandomForestRegressor(random_state=42, n_jobs=-1),
            param_grid=rf_param_grid,
            cv=3,
            scoring='neg_root_mean_squared_error',
            verbose=1
        )

        rf_grid.fit(X_train_scaled, y_train)
        rf_best = rf_grid.best_estimator_
        y_val_pred_gd_rf = rf_best.predict(X_val_scaled)
        y_test_pred_gd_rf = rf_best.predict(X_test_scaled)

        rf_rmse_gd_val = mean_squared_error(y_val, y_val_pred_gd_rf, squared=False)
        rf_rmse_gd_test = mean_squared_error(y_test, y_test_pred_gd_rf, squared=False)
        test_pair['rf_gd'] = y_test_pred_gd_rf
        
        # Log parameters and metrics
        mlflow.set_tag("model", "Rf_GridSearch")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)
        mlflow.log_params(ridge_gs.best_params_)
        mlflow.log_metric("val_rmse", rf_rmse_gd_val)
        mlflow.log_metric("test_rmse", rf_rmse_gd_test)
        
        mlflow.sklearn.log_model(rf_best, artifact_path="model")
        
        pd.DataFrame({'pred': y_test_pred_gd_rf}).to_csv("y_test_pred_gd_rf.csv", index=False)
        mlflow.log_artifact("y_test_pred_gd_rf.csv")

    
    # -------- Random Forest + Hyperopt --------
    
    with mlflow.start_run(run_name=f"RF_HO_store_{store_id}_prod_{product_id}"):
        
        def objective(params):
            rf = RandomForestRegressor(
                n_estimators=int(params['n_estimators']),
                max_depth=int(params['max_depth']),
                min_samples_split=int(params['min_samples_split']),
                min_samples_leaf=int(params['min_samples_leaf']),
                random_state=42,
                n_jobs=-1
            )
            rf.fit(X_train_scaled, y_train)
            pred = rf.predict(X_val_scaled)
            loss = mean_squared_error(y_val, pred, squared=False)
            return {'loss': loss, 'status': STATUS_OK}

        space = {
            'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
            'max_depth': hp.quniform('max_depth', 3, 15, 1),
            'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1),
            'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1)
        }

        trials = Trials()
        best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=25, trials=trials)

        rf_best = RandomForestRegressor(
            n_estimators=int(best_params['n_estimators']),
            max_depth=int(best_params['max_depth']),
            min_samples_split=int(best_params['min_samples_split']),
            min_samples_leaf=int(best_params['min_samples_leaf']),
            random_state=42,
            n_jobs=-1
        )
        rf_best.fit(X_train_scaled, y_train)
        y_val_pred_ho_rf = rf_best.predict(X_val_scaled)
        y_test_pred_ho_rf = rf_best.predict(X_test_scaled)
        rf_rmse_ho_val = mean_squared_error(y_val, y_val_pred_ho_rf, squared=False)
        rf_rmse_ho_test = mean_squared_error(y_test, y_test_pred_ho_rf, squared=False)
        test_pair['rf_ho'] = y_test_pred_ho_rf
        
        # Log parameters and metrics
        mlflow.set_tag("model", "Rf_Hyperopt")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)
        mlflow.log_params(ridge_gs.best_params_)
        mlflow.log_metric("val_rmse", rf_rmse_ho_val)
        mlflow.log_metric("test_rmse", rf_rmse_ho_test)
        
        mlflow.sklearn.log_model(rf_best, artifact_path="model")
        
        pd.DataFrame({'pred': y_test_pred_ho_rf}).to_csv("y_test_pred_ho_rf.csv", index=False)
        mlflow.log_artifact("y_test_pred_ho_rf.csv")
    
    # -------- Lightgbm + Hyperopt --------
    
    with mlflow.start_run(run_name=f"LightGBM_store_{store_id}_prod_{product_id}"):
        lgb_train = lgb.Dataset(X_train_scaled, label=y_train)
        lgb_val = lgb.Dataset(X_val_scaled, label=y_val)

        space = {
            'num_leaves': hp.quniform('num_leaves', 20, 150, 1),
            'max_depth': hp.quniform('max_depth', 3, 15, 1),
            'learning_rate': hp.loguniform('learning_rate', np.log(0.005), np.log(0.2)),
            'min_child_samples': hp.quniform('min_child_samples', 10, 100, 1),
            'subsample': hp.uniform('subsample', 0.6, 1.0),
            'colsample_bytree': hp.uniform('colsample_bytree', 0.6, 1.0),
        }

        def objective(params):
            params = {
                'boosting_type': 'gbdt',
                'objective': 'regression',
                'metric': 'rmse',
                'verbose': -1,
                'seed': 42,
                'num_leaves': int(params['num_leaves']),
                'max_depth': int(params['max_depth']),
                'learning_rate': params['learning_rate'],
                'min_child_samples': int(params['min_child_samples']),
                'subsample': params['subsample'],
                'colsample_bytree': params['colsample_bytree'],
                'feature_pre_filter': 'false'
            }

            model = lgb.train(
                params,
                lgb_train,
                valid_sets=[lgb_train, lgb_val],
                valid_names=['train', 'valid'],
                num_boost_round=1000,
                callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
            )

            preds = model.predict(X_val_scaled, num_iteration=model.best_iteration)
            rmse = mean_squared_error(y_val, preds, squared=False)
            return {'loss': rmse, 'status': STATUS_OK}

        trials = Trials()
        best_params = fmin(
            fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=30,
            trials=trials,
            rstate=np.random.default_rng(42)
        )

        best_params['num_leaves'] = int(best_params['num_leaves'])
        best_params['max_depth'] = int(best_params['max_depth'])
        best_params['min_child_samples'] = int(best_params['min_child_samples'])
        best_params.update({
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': 'rmse',
            'verbose': -1,
            'seed': 42
        })

        final_model = lgb.train(
            best_params,
            lgb_train,
            valid_sets=[lgb_train, lgb_val],
            valid_names=['train', 'valid'],
            num_boost_round=1000,
            callbacks=[lgb.early_stopping(50), lgb.log_evaluation(100)]
        )

        y_val_pred_lgb = final_model.predict(X_val_scaled, num_iteration=final_model.best_iteration)
        y_test_pred_lgb = final_model.predict(X_test_scaled, num_iteration=final_model.best_iteration)
        lgb_rmse_val = mean_squared_error(y_val, y_val_pred_lgb, squared=False)
        lgb_rmse_test = mean_squared_error(y_test, y_test_pred_lgb, squared=False)
        test_pair['lgb'] = y_test_pred_lgb
        
        # Log params and metrics
        mlflow.set_tag("model", "LightGBM")
        mlflow.set_tag("store_id", store_id)
        mlflow.set_tag("product_id", product_id)
        mlflow.log_params(best_params)
        mlflow.log_metric("val_rmse", lgb_rmse_val)
        mlflow.log_metric("test_rmse", lgb_rmse_test)

        # Log model
        mlflow.lightgbm.log_model(final_model, artifact_path="model")

        # Optionally log predictions
        pd.DataFrame({'pred': y_test_pred_lgb}).to_csv("y_test_pred_lgb.csv", index=False)
        mlflow.log_artifact("y_test_pred_lgb.csv")
    

    # Save predictions and metrics
    results.append({
        'store': store_id,
        'product': product_id,
        'ridge_rmse_gd_val': ridge_rmse_gd_val,
        'ridge_rmse_gd_test': ridge_rmse_gd_test,
        'ridge_rmse_ho_val': ridge_rmse_ho_val,
        'ridge_rmse_ho_test': ridge_rmse_ho_test,
        'rf_rmse_gd_val': rf_rmse_gd_val,
        'rf_rmse_gd_test': rf_rmse_gd_test,
        'rf_rmse_ho_val': rf_rmse_ho_val,
        'rf_rmse_ho_test': rf_rmse_ho_test,
        'lgb_rmse_val': lgb_rmse_val,
        'lgb_rmse_test': lgb_rmse_test
    })
    
    prediction_test.append(test_pair)

# Convert results to DataFrame for inspection
results_df = pd.DataFrame(results)
prediction_df = pd.concat(prediction_test)

mlflow.set_experiment("Final Aggregated Forecasting")

with mlflow.start_run(run_name="Overall RMSE Summary Test Regression"):
    for i in prediction_df.columns[-5:]:
        print(i)
        print('RMSE: ', mean_squared_error(prediction_df['number_sold'], prediction_df[i], squared=False))
        print()
        
        rmse = mean_squared_error(prediction_df['number_sold'], prediction_df[i], squared=False)
        
        # Log each model’s global RMSE
        mlflow.set_tag("model", i)
        mlflow.log_metric(f"{i}_rmse", rmse)
        
        # Optional: log the full predictions dataframe
        prediction_df[['Date', 'store', 'product', 'number_sold', i]].to_csv(f"final_predictions_{i}.csv", index=False)
        mlflow.log_artifact(f"final_predictions_{i}.csv")

0it [00:00, ?it/s]

2025/07/11 09:30:33 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/07/11 09:30:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instea


100%|███████████████████████████████████████████████| 30/30 [00:00<00:00, 187.66trial/s, best loss: 11.558420038902158]


2025/07/11 09:31:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 09:31:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Fitting 3 folds for each of 36 candidates, totalling 108 fits


...


  0%|                                                                           | 0/30 [00:00<?, ?trial/s, best loss=?]
                                                                                                                    Training until validation scores don't improve for 50 rounds

  0%|                                                                           | 0/30 [00:00<?, ?trial/s, best loss=?]
                                                                                                                    Early stopping, best iteration is:
[328]	train's rmse: 10.2918	valid's rmse: 10.6296

  3%|█▋                                                | 1/30 [00:00<00:05,  5.62trial/s, best loss: 10.62963190026999]
                                                                                                                    Training until validation scores don't improve for 50 rounds

  3%|█▋                                                | 1/30 [00:00<00:05,  5.62

 37%|█████████████████▌                              | 11/30 [00:03<00:04,  4.43trial/s, best loss: 10.529273220900423]
                                                                                                                    Training until validation scores don't improve for 50 rounds

 37%|█████████████████▌                              | 11/30 [00:03<00:04,  4.43trial/s, best loss: 10.529273220900423]
                                                                                                                    Early stopping, best iteration is:
[566]	train's rmse: 10.171	valid's rmse: 10.6145

 40%|███████████████████▏                            | 12/30 [00:03<00:05,  3.36trial/s, best loss: 10.529273220900423]
                                                                                                                    Training until validation scores don't improve for 50 rounds

 40%|███████████████████▏                            | 12/30 [00:03<00:05,  3.36tri

 73%|███████████████████████████████████▏            | 22/30 [00:07<00:02,  3.11trial/s, best loss: 10.526732854142415]
                                                                                                                    Training until validation scores don't improve for 50 rounds

 73%|███████████████████████████████████▏            | 22/30 [00:08<00:02,  3.11trial/s, best loss: 10.526732854142415]
                                                                                                                    Early stopping, best iteration is:
[919]	train's rmse: 10.4516	valid's rmse: 10.6073

 77%|████████████████████████████████████▊           | 23/30 [00:08<00:02,  3.24trial/s, best loss: 10.526732854142415]
                                                                                                                    Training until validation scores don't improve for 50 rounds

 77%|████████████████████████████████████▊           | 23/30 [00:08<00:02,  3.24tr

2025/07/11 11:29:00 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/11 11:29:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


ridge_gd
RMSE:  55.10128661396115

ridge_ho
RMSE:  45.060792533629645

rf_gd
RMSE:  44.666798943656154

rf_ho
RMSE:  43.87234233596049

lgb
RMSE:  38.78252935406796

